## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

In [0]:
python -m ensurepip --default-pip

SyntaxError: invalid syntax (<ipython-input-4-99e4d36e16e3>, line 1)

In [0]:
!pip install keras

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/prices.csv')

### Check all columns in the dataset

In [0]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(columns="date") 
data = data.drop(columns="symbol") 

In [0]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data['volume_trans'] = data['volume']/1000000
data[:1000]

,open,close,low,high,volume,volume_trans
0,123.430000,125.839996,122.309998,126.250000,2163600.0,2.1636
1,125.239998,119.980003,119.940002,125.540001,2386400.0,2.3864
2,116.379997,114.949997,114.930000,119.739998,2489500.0,2.4895
3,115.480003,116.620003,113.500000,117.440002,2006300.0,2.0063
4,117.010002,114.970001,114.089996,117.330002,1408600.0,1.4086
5,115.510002,115.550003,114.500000,116.059998,1098000.0,1.0980
6,116.459999,112.849998,112.589996,117.070000,949600.0,0.9496
7,113.510002,114.379997,110.050003,115.029999,785300.0,0.7853
8,113.330002,112.529999,111.919998,114.879997,1093700.0,1.0937
9,113.660004,110.379997,109.870003,115.870003,1523500.0,1.5235


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X = data.drop("volume_trans" , axis=1)
y = data.pop("volume_trans")
train_set, test_set, train_labels, test_labels = train_test_split(X, y, test_size=.30, random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np
train_set = np.float32(train_set)
test_set = np.float32(test_set)
train_labels = np.float32(train_labels)
test_labels = np.float32(test_labels)


### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(train_set)
transformer
train_transform=transformer.transform(train_set)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[5,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

Instructions for updating:
Colocations handled automatically by placer.


2.Define a function to calculate prediction

In [0]:
y = tf.add(tf.matmul(train_transform,W),b,name='output')

y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')3.Loss (Cost) Function [Mean square error]

In [0]:
x = tf.placeholder(shape=[None,5],dtype=tf.float32, name='x-input')
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:train_transform, y_:y})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

### Get the shapes and values of W and b

### Model Prediction on 1st Examples in Test Dataset

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
iris = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Iris.csv')

In [0]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
categorial_features = ['Species']

for categorical_feature in list(iris.columns):
    if categorical_feature in categorial_features:
        iris[categorical_feature] = iris[categorical_feature].astype('category')
        iris_with_dummies = pd.get_dummies(iris, sparse=True)
        label_encoder = LabelEncoder()
iris_encoded = label_encoder.fit_transform(values)


In [0]:
import numpy as np 
import pandas as pd
from sklearn import preprocessing 
iris['Species'].unique() 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
iris['Species']= label_encoder.fit_transform(iris['Species']) 
  
iris['Species'].unique()


array([0, 1, 2])

In [0]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


### Splitting the data into feature set and target set

In [0]:
from sklearn.model_selection import train_test_split
X = iris.drop("Species" , axis=1)
y = iris.pop("Species")

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
model = Sequential()
#Adding 3 dense layers
model.add(Dense(1, input_dim=5,init='uniform'))
model.add(Dense(5,init='uniform'))
model.add(Dense(3,init='uniform'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# Fit the model
model.fit(X, y, nb_epoch=20, batch_size=10,  verbose=2)
# evaluate the model

scores = model.evaluate(X, y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

### Model Training 

### Model Prediction

### Save the Model

In [0]:
model.save('iris.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?